# A Python bases script to read in credit card reviews from `cardraitings.com`, collect reward information from those credit cards, and try to find the best combination of cards for an individual's expenses

In [1]:
import os
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import pickle


In [2]:
# Opening us the website that contains links to reviews for 
# the cedit cards we will be investiating

parent_url = 'https://www.cardratings.com/credit-card-list.html'
res = requests.get(parent_url)
html_page = res.content
soup = BeautifulSoup(html_page, 'html.parser')
cc_urls = []
for link in soup.find_all('a', href=True):
    if 'cardratings.com/credit-card/' in link.get('href'):
        print(link.get('href'))
        cc_urls.append(link.get('href'))


https://www.cardratings.com/credit-card/aboc-platinum-rewards-credit-card.html
https://www.cardratings.com/credit-card/american-airlines-aadvantage-mileup-card.html
https://www.cardratings.com/credit-card/american-express-cash-magnet-card.html
https://www.cardratings.com/credit-card/american-express-gold-card.html
https://www.cardratings.com/credit-card/american-express-green-card
https://www.cardratings.com/credit-card/avianca-vida-visa-card.html
https://www.cardratings.com/credit-card/avianca-vuela-visa-card.html
https://www.cardratings.com/credit-card/aaa-member-rewards-visa-signature-card.html
https://www.cardratings.com/credit-card/aacs-american-association-of-christian-schools-visa-credit-card-review
https://www.cardratings.com/credit-card/aarp-visa-signature-card.html
https://www.cardratings.com/credit-card/aeo-inc.-credit-card.html
https://www.cardratings.com/credit-card/aeo-inc-visa-card
https://www.cardratings.com/credit-card/abercrombie-fitch-credit-card
https://www.cardrati

In [3]:
def get_rewards_from_string(string):
    """
    A function to read through a string and pick out the rewards
    for specific categories based on associated words. These 
    associated word may be incomplete.
    
    Inputs:
    - string (str), a string containing the rewards for a credit card
    
    Outputs:
    - rewards (dict), a dictionary with keys of categories and values
        of points earned per dollar spent
    """
    key_words = {
        'flights':['flights', 'airlines','travel', 'air', 'southwest', 'fly'],
        'hotel': ['travel', 'hotel'] ,
        'grocery':['supermarket', 'grocery', 'groceries'],
        'gas':['station', 'gas'],
        'utilities':['telephone', 'shipping', 'internet', 'cabel'],
        'restaurants':['restaurants', 'dining'],
        'other':['select', 'rotating']
    }
    rewards_dict = {}
    all_15 =  False
    if not isinstance(string, str):
        # a catch all incase if string fed in is faulty
        rewards = {
            'flights':0,
            'hotel':0,
            'grocery':0,
            'gas':0,
            'utilities':0,
            'restaurants':0,
            'other':0 
        }
        return rewards
        
    for category, key_word_list in key_words.items():     
        for sentence in string.replace('U.S.', 'US').split('. '):
            if any(map(lambda x: x in sentence.lower(), key_word_list)):
                try:
                    multiplyer = float(re.search('(\d+(?:\.\d+)?)', sentence).group())
                    if multiplyer > 15:
                        # This isn't a reward point value
                        continue
                    if multiplyer == 1.5:
                        # Cards with a 1.5 multiplier are often 1.5% 
                        # cash back at everything
                        all_15 =  True
                    rewards_dict[category] = multiplyer
                except:
                    continue
        if category not in rewards_dict.keys():
            rewards_dict[category] = 0
    
    if all_15:
        # This is a 1.5% back on all purchases card
        for category in rewards_dict:
            rewards_dict[category] = 1.5
    if any(map(lambda x: x >= 1, rewards_dict.values())):
        # Cards with any rewards are assumed to have 1 point back
        # in all other categories, setting rewards for everything else to 1
        for category in rewards_dict.keys():
            if rewards_dict[category] == 0:
                rewards_dict[category] = 1
        
    return rewards_dict

def get_annual_fee(string):
    """
    A function to find the annual fee (or dollar amount) in a string
    
    Inputs:
    - string (str), a string containing a $dollar amount
    
    Output (float), the dollar amount of the annual fee
    """
    try:
        return float(re.search('\$\d+', string).group()[1:])
    except AttributeError:
        return 0.
    
def get_annual_bonus(string):
    """
    A function to look through a string and try and find the annual 
    bonuses that you get with a particular card that offsets the annual
    fee of those cards
    
    Inputs:
    - string (str), a string that contians a description of the card bonuses
    
    Outputs:
    - credits (float), the dollar amount of the annual bonus
    """
    credits = 0
    for line in string.split('\n'):
        credit_words = ['saving', 'credits'] # we want to see these words in a sentence
        comparitive_words = ['higher', 'lower', 'worse', 'better'] # we don't want to see these words
        if any(map(lambda x: x in line.lower(), credit_words)) and not any(map(lambda x: x in line, comparitive_words)):
            try:
                credits += max([float(credit[1:]) for credit in re.findall('\$\d+', line) if float(credit[1:]) >= 50])
            except ValueError:
                pass
    return credits

def get_rewards_info_from_url(url):
    """
    A function that will request the html data from a url and find
    the rewards, annual fee, and annual bonuses for that link. This
    function isn't 100% working (issues with annual fee and bonus). But
    can parse out the rewards.
    
    Inputs:
    - url (str), a string representation of the url
    
    Outputs:
    - title (str), the title of the post
    - rewards (dict), a dictionary containing the rewards, annual_fee
        annual_bonus, and net_fee
    """
    
    print()
    print()
    res = requests.get(url)
    html_page = res.content
    soup = BeautifulSoup(html_page, 'html.parser')
    title = soup.title.text
    print(title)
    print(url)
    if '400' in title:
        rewards = {
            'flights':0,
            'hotel':0,
            'grocery':0,
            'gas':0,
            'utilities':0,
            'restaurants':0,
            'other':0 
        }
        rewards['annual_fee'] = 0
        rewards['annual_bonus'] = 0
        rewards['net_fee'] = 0
        return None, rewards
        
    d = soup.find('div', itemprop='description')
    try:
        df = pd.read_html(d.decode())[0].T.set_index(0)
    except:
        rewards = {
            'flights':0,
            'hotel':0,
            'grocery':0,
            'gas':0,
            'utilities':0,
            'restaurants':0,
            'other':0 
        }
        rewards['annual_fee'] = 0
        rewards['annual_bonus'] = 0
        rewards['net_fee'] = 0
        return title, rewards
        
        
    if 'Rewards' in df.index:
        rewards = get_rewards_from_string(df[1]['Rewards'])
        try:
            rewards['annual_fee'] = get_annual_fee(df[1]['Annual Fee'])
        except KeyError:
            rewards['annual_fee'] = 0
            for line in d.text.split('\n'):
                if 'annual fee' in line.lower():
                    rewards['annual_fee'] = get_annual_fee(line)
                    break
            
            
        rewards['annual_bonus'] = get_annual_bonus(d.text)
        rewards['net_fee'] = rewards['annual_fee'] - rewards['annual_bonus']
    else:
        rewards = get_rewards_from_string(d.text)
        rewards['annual_fee'] = 0
        rewards['annual_bonus'] = 0
        rewards['net_fee'] = 0
    return title, rewards
    

In [4]:
if not os.path.exists('rewards.pkl'): 
    # If we've run this before, we're gonna read in a pickle file
    # to save time
    total_rewards = {}
    for cc_url in cc_urls:
        if cc_url == "https://www.cardratings.com/credit-card/connect-classic":
            continue # known to be broken url
        title, rewards = get_rewards_info_from_url(cc_url)
        if title: # cleaning up the title
            title_string = ''
            review = False
            for t in title.split():
                if '-' in t.lower() or 'review' in t.lower():
                    break
                title_string += t + ' '
            total_rewards[title_string.strip()] = rewards

    # Making and saving the dataframe
    rewards_df = pd.DataFrame(total_rewards).T
    f = open('rewards.pkl', 'wb')
    pickle.dump(rewards_df, f)
else:
    print('Reading results from a previous pickle file')
    f = open('rewards.pkl', 'rb')
    rewards_df = pickle.load(f)
    
    
rewards_df

Reading results from a previous pickle file


,flights,hotel,grocery,gas,utilities,restaurants,other,annual_fee,annual_bonus,net_fee
ABOC Platinum Rewards Mastercard Credit Card,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0
American Airlines AAdvantage MileUp℠ Card,2.0,1.0,2.0,1.0,1.0,1.0,1.0,0.0,500.0,-500.0
American Express Cash Magnet Card,1.5,1.5,1.5,1.5,1.5,1.5,1.5,0.0,0.0,0.0
American Express® Gold Card,3.0,3.0,4.0,1.0,1.0,4.0,1.0,250.0,220.0,30.0
American Express® Green Card,3.0,3.0,1.0,1.0,1.0,3.0,1.0,150.0,200.0,-50.0
...,...,...,...,...,...,...,...,...,...,...
Zions Bank AmaZing Cash™ for Business,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0
Zions Bank AmaZing Rate™ for Business,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zions Bank AmaZing Rate™ Credit Card,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zions Bank AmaZing Rewards® Credit Card,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
# Currently don't trust the annual fee and bonus section, 
# so we're going to process based on the rewards alone

# TODO: Find accurate net_fee for all cards and use that when 
# calculating rewards
to_plot = rewards_df[rewards_df.columns[:-3]]

# Only looking at credit card with greater than 1% back at everything
to_plot = to_plot[to_plot.sum(axis=1) > len(to_plot.columns)]

# Removing cards that have a weirdly high point value
# These data are either parsed incorrectly or are hotel rewards were
# the point to cent ratio is high (e.g. many points to a cent)
to_plot = to_plot[to_plot.sum(axis=1) <= 2*len(to_plot.columns)] 
to_plot

,flights,hotel,grocery,gas,utilities,restaurants,other
American Airlines AAdvantage MileUp℠ Card,2.0,1.0,2.0,1.0,1.0,1.0,1.0
American Express Cash Magnet Card,1.5,1.5,1.5,1.5,1.5,1.5,1.5
American Express® Green Card,3.0,3.0,1.0,1.0,1.0,3.0,1.0
Avianca Vuela Visa Card,1.0,1.0,2.0,2.0,1.0,1.0,1.0
AAA® Member Rewards Visa Signature® Card,1.0,1.0,2.0,2.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...
Webster Bank Visa® Business Rewards PLUS Card,1.5,1.5,1.5,1.5,1.5,1.5,1.5
Wells Fargo Business Elite Card,1.5,1.5,1.5,1.5,1.5,1.5,1.5
Wells Fargo Business Platinum Credit Card,1.5,1.5,1.5,1.5,1.5,1.5,1.5
Wells Fargo Business Secured Credit Card,1.5,1.5,1.5,1.5,1.5,1.5,1.5


In [6]:
# From my own annual expenses over the last 12 months

# Reading in a CSV file from my mint.com accout
transactions = pd.read_csv('transactions-3.csv')
transactions['Date'][0]
def compare_date(date):
    """
    A function to find if a date is within a year from now
    
    Inputs:
    - date (str), a date in the mm/dd/yyyy format
    
    Returns:
    - bool (str), a bool indicating if the date is 
        within a year from now or not
    """
    other = '4/3/2020'
    m,d,y = date.split('/')
    om,od,oy = other.split('/')
    m = int(m)
    d = int(d)
    y = int(y)
    om = int(om)
    od = int(od)
    oy = int(oy)
    if oy <= y+1:
        if om < m:
            return True
        elif om == m:
            if od < d:
                return True
            else:
                return False
        else: 
            return False
    else:
        return False
# Removing all positive transactions
transactions = transactions[transactions['Transaction Type'] == 'debit']

# Finding all transactions within a year of now
transactions = transactions[transactions['Date'].map(compare_date)]

# Parsing out annual expenses for each specific category
food = transactions[transactions.Category.map(lambda x: x.lower() in ['food & dinging', 'alcohol & bars', 'restaurants', 'fast food'])].Amount.sum()
flights = transactions[transactions.Category.map(lambda x: x.lower() in ['air travel'])].Amount.sum()
hotel = transactions[transactions.Category.map(lambda x: x.lower() in ['hotel'])].Amount.sum()
gas = transactions[transactions.Category.map(lambda x: x.lower() in ['auto & transport', 'gas & fuel'])].Amount.sum()
grocery = transactions[transactions.Category.map(lambda x: x.lower() in ['groceries'])].Amount.sum()
utilities = transactions[transactions.Category.map(lambda x: x.lower() in ['bills & utilities', 'mobile phone', 'internet'])].Amount.sum()
other = transactions.Amount.sum() - food -  flights - hotel - gas - grocery

# Making a dict containing all of the expenses, to be used later
expenses = {
    'flights': flights,
    'hotel':hotel,
    'grocery':grocery,
    'gas':gas,
    'utilities':utilities,
    'restaurants': food,
    'other':other
    
}

expenses

{'flights': 895.4899999999999,
 'hotel': 9.9,
 'grocery': 1803.92,
 'gas': 336.03999999999996,
 'utilities': 0.0,
 'restaurants': 4069.8500000000004,
 'other': 44067.240000000005}

In [7]:
def calculate_rewards(rewards_df, expenses):
    """
    A function to calculate the max rewards for a parsed dataframe.
    TODO: this needs to eventually include net_fee in it's equation.
    
    Inputs:
    - rewards_df (pd.DateFrame), a sliced dataframe with columns
        similar to to_plot
    - expenses (dict), a dictionary containing all of a person's 
        annual expenses
        
    Returns:
    - cash_rewards (float), the annual rewards one would recieve
    """
    cash_rewards = 0
    for category, expense in expenses.items():
        try:
            if isinstance(rewards_df[category], float):
                rate = rewards_df[category] / 100
            else:
                rate = max(rewards_df[category]) / 100
            cash_rewards += rate * expense
        except:
            continue
        
    return cash_rewards

In [19]:
# Given that there are many different combinations of cards, we're 
# going to be solving this stochasticly. 1000 random combination of
# `card_number` of cards will be chosen and will be used to estimate 
# annual rewards. These will be stored to find an approximate max, median
# and min amount of rewards for each number_of_cards.

if os.path.exists('calculated_rewards.pkl') and os.path.exists('cloud.pkl'):
    f = open('calculated_rewards.pkl', 'rb')
    calculated_rewards = pickle.load(f)
    f = open('cloud.pkl', 'rb')
    word_cloud = pickle.load(f)
else:
    calculated_rewards = {} # to keep track of total rewards
    word_cloud = {} # to be used for a wordcloud
    for card_number in range(11):
        # Between 0 and 10 cards
        card_rewards = []
        cloud_list = []

        for i in range(1000):
            choice = np.random.choice(to_plot.index, card_number)
            card_rewards.append(calculate_rewards(to_plot.loc[choice], expenses))
            cloud_list.append((calculate_rewards(to_plot.loc[choice], expenses), choice))
        calculated_rewards[card_number] = card_rewards
        word_cloud[card_number] = cloud_list

    f = open('calculated_rewards.pkl', 'wb')
    pickle.dump(calculated_rewards, f)
    f = open('cloud.pkl', 'wb')
    pickle.dump(word_cloud, f)    